In [ ]:
import os
import cv2
import dlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [ ]:
print(tf.__version__)

2.7.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def plot_hist(hist):
    plt.figure(figsize=(20,20))
    fig, axs = plt.subplots(2)
    
    axs[0].plot(hist.history["accuracy"], label="Train accuracy")
    axs[0].plot(hist.history["val_accuracy"], label="Test accuracy")    
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    axs[1].plot(hist.history["loss"], label="Train error")
    axs[1].plot(hist.history["val_loss"], label="Test error")    
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_cf_matrix(cf_matrix, title):
    ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
    ax.set_title(title);
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');
    #ax.xaxis.set_ticklabels(genres, rotation=90)
    #ax.yaxis.set_ticklabels(genres, rotation=0)
    plt.show()

In [ ]:
!unzip /content/drive/MyDrive/casia_10_split.zip -d .

In [ ]:
path_train = "./casia_10_split/train/"
path_test = "./casia_10_split/test/"
leaders_train = os.listdir(path_train)
leaders_test = os.listdir(path_test)
leaders_train.sort()
leaders_test.sort()
#print(leaders_train)
#print(leaders_test)
print(leaders_train==leaders_test)

True


In [ ]:
#Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
# Load VGG Face model weights
model.load_weights('./drive/MyDrive/lfw/vgg_face_weights.h5')

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model)

In [ ]:
# Remove Last Softmax layer and get model upto last flatten layer with outputs 2622 units
vgg_face = Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

In [ ]:
#Prepare Train Data
x_train = []
y_train = []
person_folders = os.listdir("./casia_10_split/train/")
person_rep = dict()
for i, person in enumerate(leaders_train):
  print(person, i)
  person_rep[person] = i
  image_names = os.listdir("./casia_10_split/train/" + person + '/')
  for image_name in image_names:
    #print(image_name)
    img = load_img("./casia_10_split/train/" + person + '/' + image_name, target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = preprocess_input(img)
    img_encode = vgg_face(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)

0000099 0
0000100 1
0000102 2
0000103 3
0000105 4
0000107 5
0000108 6
0000114 7
0000117 8
0000119 9


In [ ]:
person_rep

{'0000099': 0,
 '0000100': 1,
 '0000102': 2,
 '0000103': 3,
 '0000105': 4,
 '0000107': 5,
 '0000108': 6,
 '0000114': 7,
 '0000117': 8,
 '0000119': 9}

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_train[0])
print(y_train[0])

(960, 2622) (960,)
[ 1.23979294  3.42265272 -1.12267327 ... -3.59475088  2.88625431
  6.1710577 ]
0


In [ ]:
#Prepare Test Data
x_test=[]
y_test=[]
person_folders = os.listdir("./casia_10_split/test/")
for i, person in enumerate(leaders_train):
  print(person, person_rep.get(person))
  image_names = os.listdir("./casia_10_split/test/" + person + '/')
  for image_name in image_names:
    #print(image_name)
    img = load_img("./casia_10_split/test/" + person + '/' + image_name, target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = preprocess_input(img)
    img_encode = vgg_face(img)
    x_test.append(np.squeeze(K.eval(img_encode)).tolist())
    #y_test.append(person_rep.get(person))
    y_test.append(person_rep.get(person))

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
print(x_test.shape, y_test.shape)
print(x_test[0])
print(y_test[0])

(417, 2622) (417,)
[ 1.91138816  0.58899343 -0.19249925 ... -3.26062417  1.52417946
  3.62644005]
0


In [ ]:
# Softmax regressor to classify images based on encoding 
classifier_model=Sequential()
classifier_model.add(Dense(units=100,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.3))
classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=len(leaders_test),kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))
classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='nadam',metrics=['accuracy'])

In [ ]:
hist = classifier_model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
30/30 [==============================] - 2s 12ms/step - loss: 1.0985 - accuracy: 0.7729 - val_loss: 0.6118 - val_accuracy: 0.9424
Epoch 2/100
30/30 [==============================] - 0s 6ms/step - loss: 0.7456 - accuracy: 0.9260 - val_loss: 0.4794 - val_accuracy: 0.9568
Epoch 3/100
30/30 [==============================] - 0s 7ms/step - loss: 0.6363 - accuracy: 0.9521 - val_loss: 0.4365 - val_accuracy: 0.9640
Epoch 4/100
30/30 [==============================] - 0s 6ms/step - loss: 0.5607 - accuracy: 0.9615 - val_loss: 0.3934 - val_accuracy: 0.9688
Epoch 5/100
30/30 [==============================] - 0s 6ms/step - loss: 0.4851 - accuracy: 0.9719 - val_loss: 0.3741 - val_accuracy: 0.9640
Epoch 6/100
30/30 [==============================] - 0s 6ms/step - loss: 0.4451 - accuracy: 0.9698 - val_loss: 0.3473 - val_accuracy: 0.9544
Epoch 7/100
30/30 [==============================] - 0s 7ms/step - loss: 0.3852 - accuracy: 0.9823 - val_loss: 0.3140 - val_accuracy: 0.9640
Epoch 8/100


In [ ]:
plot_hist(hist)

In [ ]:
test_error, test_accuracy = classifier_model.evaluate(x_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy}")

y_pred = classifier_model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
print(y_pred.shape)

14/14 [==============================] - 0s 3ms/step - loss: 0.1593 - accuracy: 0.9616
Test accuracy: 0.9616307020187378
(417, 10)
